In [189]:
import math
import numpy as np
import scipy
import scipy.optimize
import matplotlib.pyplot as plt
import csv

In [207]:
#Helper Functions

def exist(pts, latt):
    latt = np.array(latt)
    for i in range(pts.shape[0]):
        if pts[i][0]==latt[0]:
            if pts[i][1]==latt[1]:
                if pts[i][2]==latt[2]:
                    return 1
    return 0

def face_list(p1):
    face_p1 = p1.faces(1)
    list_face_p1 = list(face_p1)
    faces = []
    for i in range(len(face_p1)):
        faces.append([])
    for i in range(len(face_p1)):
        faces[i].append(list(list_face_p1[i].vertices()[0]))
        faces[i].append(list(list_face_p1[i].vertices()[1]))
    return faces

def vert_to_list(vert):
    num_vert = len(vert)
    vert_list = []
    for i in range(num_vert):
        vert_list.append(list(vert[i]))
    return vert_list

def dist(p1, p2):
    return sqrt((p1[0]-p2[0])^2+(p1[1]-p2[1])^2+(p1[2]-p2[2])^2)

def on_edge(latt, faces):
    for i in range(len(faces)):
        if (dist(faces[i][0], faces[i][1]) == (dist(faces[i][0], latt) + dist(faces[i][1], latt))):
            #print 'edge: ', faces[i][0], ' and ', faces[i][1]
            #print 'l1: ', dist(faces[i][0], faces[i][1])
            #print 'l2: ', dist(faces[i][0], latt) + dist(faces[i][1], latt)
            return 1
    return 0

#Add lattice points onto the edge of a polygon
def add_lattice(poly):
    #convert vertices to points
    pts = []
    vert = list(poly.vertices())
    num_pts = len(vert)
    
    for i in range(num_pts):
        pts.append(list(vert[i]))
        
    #find the maximum of points
    pts = np.array(pts)
    pts_max = int(max(np.amax(np.absolute(pts), axis=0)))+1
    pts_new = pts
    
    faces = face_list(poly)
    
    for i in range(-pts_max, pts_max):
        for j in range(-pts_max, pts_max):
            for k in range(-pts_max, pts_max):
                latt = [i,j,k]
                if latt in pts.tolist():
                    continue
                if poly.contains(latt) == 1 or on_edge(latt, faces) == 1:
                #if on_edge(latt, faces) == 1:
                    pts_new = np.append(pts_new, np.array(latt).reshape((1,3)), axis = 0)  
    #print 'pts_new: '
    #print pts_new
    pts_new = pts_new.tolist()
    poly_new = Polyhedron(vertices = pts_new)
    
    
    
    return poly_new, pts_new


#remove a point from a polyhedron
#input: points of a polyhedron; a point to be removed
#output: polyhedron with a point removed
def remove_pts(pts, remove_pt):
    #points to remove
    pts_removed = pts
    
    #backup points
    pts_save = np.array(pts)
    
    #remove points
    pts_removed.remove(remove_pt)
    
    #restore
    pts = pts_save.tolist()
    
    poly_new = Polyhedron(vertices = pts_removed)
    
    return poly_new

#Count the number of lattice points inside a polytope
def num_latt(poly):
    count = 0
    
    #convert vertices to points
    pts = []
    vert = list(poly.vertices())
    num_pts = len(vert)
    
    for i in range(num_pts):
        pts.append(list(vert[i]))
    
    #find the maximum of points
    pts = np.array(pts)
    pts_max = int(max(np.amax(pts, axis=0)))+1
    
    faces = face_list(poly)
    
    for i in range(-pts_max, pts_max):
        for j in range(-pts_max, pts_max):
            for k in range(-pts_max, pts_max):
                latt = [i,j,k]
                if latt in pts.tolist():
                    continue
                if poly.contains(latt) == 1:
                    count += 1
    return count

In [210]:
#Execution Cell

output_path = 'output/polygon/poly_out_2.txt'

def generate_poly(size, num_poly, output_path):
    
    output = open(output_path, 'w')
    
    p1 = polytopes.hypercube(3)
    p1 = p1.dilation(size)
    
    poly, pts = add_lattice(p1)
    print "Step 1: add lattice done."
    
    for i in range(num_poly):
        
        face_pts = list(poly.faces(2))
        num_faces = len(face_pts)
        face_idx = np.random.randint(num_faces)
        pt_idx = np.random.randint(3)
        remove_pt = list(face_pts[face_idx].vertices()[pt_idx])
        poly = remove_pts(pts, remove_pt)
        poly_vert = vert_to_list(poly.vertices())
        output.write("%s\n" % poly_vert)
        poly.plot().save("img/plot_2_%d.png" % i)
        
        if len(poly_vert) < 3:
            print 'Not enough points.'
            break
    
    print "Step 2: remove points done."
    output.close()
    
generate_poly(2, 30, output_path)

Step 1: add lattice done.
Step 2: remove points done.


In [212]:
import imageio

def generate_gif(num_poly):
    images = []
    for i in range(num_poly):
        filename = "img/plot_2_%d.png"%i
        images.append(imageio.imread(filename))
    imageio.mimsave('img/movie_2.gif', images)

generate_gif(30)

In [208]:
p1 = polytopes.hypercube(3)
p1 = p1.dilation(2)
poly, pts = add_lattice(p1)

face_pts = list(poly.faces(2))
num_faces = len(face_pts)
remove_pt = list(face_pts[0].vertices()[0])
poly = remove_pts(pts, remove_pt)

print num_latt(poly)

114
